<a href="https://colab.research.google.com/github/phelipe-sb/Rain-Prediction/blob/main/Extract_Pickle_File_from_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
path = '/content/drive/MyDrive/Weather_Project/Weather Project-6fa8e059f642.json'

In [6]:
credentials = service_account.Credentials.from_service_account_file(path)
project_id = credentials.project_id
client = bigquery.Client(project=project_id, credentials=credentials)

In [7]:
query_string = """
SELECT *
FROM (SELECT *
            , ROW_NUMBER() OVER(PARTITION BY last_updated, city ORDER BY last_updated, city) AS row_num
            FROM `weather-project-305419.Daily_Weather.Weather-Info`) as row_weather
WHERE row_num = 1
"""

In [10]:
df = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
df.head()

  last_updated_epoch      last_updated  ... extraction_date row_num
0         1619163900  2021-04-23 03:45  ...            None       1
1         1619160300  2021-04-23 07:45  ...            None       1
2         1619247600  2021-04-24 04:00  ...            None       1
3         1619263800  2021-04-24 07:30  ...            None       1
4         1619272800  2021-04-24 10:00  ...            None       1

[5 rows x 26 columns]


In [13]:
df = df.drop(columns=['row_num', 'last_updated_epoch', 'extraction_date'])

In [16]:
df.shape

(39910, 23)

In [15]:
import pickle
df.to_pickle("weather_data_30-08-2021")